# Tencent Word Embedding

本文主要介绍腾讯自研的中文词向量，包含800万个字词。

## DSG

DSG来源于腾讯AI Lab于2018年发表在ACL上的论文《Directional Skip-Gram: Explicity Distinguishing Left and Right Context for Word Embedding》，其主要思想是在Skip-Gram的基础上进一步区分上下文的左右。

回顾Skip-Gram，根据中心词预测上下文词的公式为：

$$
p(w_O|w_I) = \frac{exp({v'}_{w_O}^T v_{w_I})}{\sum_{w=1}^W exp({v'}_w^T v_{w_I})} \tag 1
$$

基于Negative Sampling优化的损失函数为：

$$
{\cal{L}} = - \log \sigma({v'}_{w_O}^Tv_{w_I}) - \sum_{i=1}^K \Bbb E_{w_i \sim P_n(w)} [\log \sigma(-{v'}_{w_i}^T v_{w_I})] \tag 2
$$

此时对损失函数求导，对于正样例数据，参数更新过程为：

$$
\begin{aligned}
\frac{\partial \cal L}{\partial {v'}_{w_O}} & = - \frac{1}{\sigma({v'}_{w_O}^Tv_{w_I})} \cdot \sigma({v'}_{w_O}^Tv_{w_I}) [1-\sigma({v'}_{w_O}^Tv_{w_I})] \cdot v_{w_I} = [\sigma({v'}_{w_O}^Tv_{w_I})-1]v_{w_I}\\
\frac{\partial \cal L}{\partial {v}_{w_I}} & = - \frac{1}{\sigma({v'}_{w_O}^Tv_{w_I})} \cdot \sigma({v'}_{w_O}^Tv_{w_I}) [1-\sigma({v'}_{w_O}^Tv_{w_I})] \cdot {v'}_{w_O} = [\sigma({v'}_{w_O}^Tv_{w_I})-1]{v'}_{w_O}
\end{aligned} \tag 3
$$

而对于负样例数据，参数更新过程为：

$$
\begin{aligned}
\frac{\partial \cal L}{\partial {v'}_{w_i}} & = - \frac{1}{\sigma(-{v'}_{w_i}^Tv_{w_I})} \cdot \sigma(-{v'}_{w_i}^Tv_{w_I}) [1-\sigma(-{v'}_{w_O}^Tv_{w_I})] \cdot (-v_{w_I}) = \sigma({v'}_{w_i}^Tv_{w_I}) \cdot v_{w_I}\\
\frac{\partial \cal L}{\partial {v}_{w_I}} & = - \frac{1}{\sigma(-{v'}_{w_i}^Tv_{w_I})} \cdot \sigma(-{v'}_{w_i}^Tv_{w_I}) [1-\sigma(-{v'}_{w_O}^Tv_{w_I})] \cdot (-{v'}_{w_i}) = \sigma({v'}_{w_i}^Tv_{w_I}) \cdot {v'}_{w_i}
\end{aligned} \tag 4
$$

DSG在Skip-Gram的基础上给每个词引入一个方向向量$\delta$，并在上面的预测公式(1)的基础上加上了方向的概率：

$$
f(w_O|w_I) = p(w_O|w_I) + g(w_O|w_I) \tag 5
$$

其中：

$$
g(w_O|w_I) = \frac{exp(\delta_{w_O}^T \delta_{w_I})}{\sum_{w_i \in V} exp(\delta_{w_i}^T \delta_{w_I})} \tag 6
$$

由此可见，方向向量$\delta$刻画了上下文词和中心词的相对位置关系（左边或者右边，二者只能选一）。以左边为例，那么对于任意上下文词，$g$越大，表示该词位于中心词的左侧，$g$越小，表示该词位于中心词的右侧。

那么在训练中，对于任何中心词，也会对其所有上下文词进行左右的训练，这就需要计算$g$的损失值，其计算过程和skip-gram的正负样例类似，有了公式(3)(4)的铺垫，我们很好理解论文直接给出的方向向量更新规则：

$$
\begin{aligned}
v_{w_t}^{(new)} & = v_{w_t}^{(old)} - \gamma(\sigma (v_{w_t}^T \delta_{w_{t+i}}) - \cal D)\delta_{w_{t+i}}\\
v_{w_{t+i}}^{(new)} & = \delta_{w_{t+i}}^{(old)} - \gamma(\sigma (v_{w_t}^T \delta_{w_{t+i}}) - \cal D)v_{w_t}
\end{aligned} \tag 7
$$

其中

$$
D = \begin{cases}
1, & i < 0\\
0, & i > 0\end{cases} \tag 8
$$

(7)式中符号稍有变化，$w_t$表示中心词，$w_{t+i}$表示上下文词，$\gamma$是学习率，所以只需要看后面的导数，它和(3)(4)式原理一样，只不过这里把$i<0$也就是左边的词看成正样本，$i>0$也就是右边的词看成负样本了。

实际使用中，发现腾讯词向量的质量还是比较高的，只不过由于文件较大，在加载和检索的时候需要拆分成小文件，从而实现分区加载。